In [ ]:
!unzip /content/drive/MyDrive/dataset/dataset.zip -d /content/drive/MyDrive/dataset/

Archive:  /content/drive/MyDrive/dataset/dataset.zip
   creating: /content/drive/MyDrive/dataset/test/heart/
   creating: /content/drive/MyDrive/dataset/test/hi/
   creating: /content/drive/MyDrive/dataset/test/pet/
  inflating: /content/drive/MyDrive/dataset/train/heart/1.jpg  
  inflating: /content/drive/MyDrive/dataset/train/heart/20220306_193202_001.jpg  
  inflating: /content/drive/MyDrive/dataset/train/heart/20220306_193202_008.jpg  
  inflating: /content/drive/MyDrive/dataset/train/heart/20220306_193202_011.jpg  
  inflating: /content/drive/MyDrive/dataset/train/heart/20220306_193202_013.jpg  
  inflating: /content/drive/MyDrive/dataset/train/heart/20220306_193202_016.jpg  
  inflating: /content/drive/MyDrive/dataset/train/heart/20220306_193202_017.jpg  
  inflating: /content/drive/MyDrive/dataset/train/heart/20220306_193202_019.jpg  
  inflating: /content/drive/MyDrive/dataset/train/heart/20220306_193202_020.jpg  
  inflating: /content/drive/MyDrive/dataset/train/heart/20220306

In [1]:
import os

train_hi_dir = os.path.join('/content/drive/MyDrive/dataset/train/hi')
train_heart_dir = os.path.join('/content/drive/MyDrive/dataset/train/heart')
train_pet_dir = os.path.join('/content/drive/MyDrive/dataset/train/pet')

train_hi_files = os.listdir(train_hi_dir)
train_heart_files = os.listdir(train_heart_dir)
train_pet_files = os.listdir(train_pet_dir)

print('total num of training hi images:', len(train_hi_files))
print('total num of training heart images:', len(train_heart_files))
print('total num of training pet images:', len(train_pet_files))

print(train_hi_files[:10])
print(train_heart_files[:10])
print(train_pet_files[:10])


total num of training hi images: 706
total num of training heart images: 765
total num of training pet images: 631
['IMG_6885.jpg', 'IMG_6883.jpg', 'IMG_6886.jpg', 'IMG_6884.jpg', 'IMG_6888.jpg', 'IMG_6889.jpg', 'IMG_6887.jpg', 'IMG_6892.jpg', 'IMG_6893.jpg', 'IMG_6891.jpg']
['KakaoTalk_20220305_205657105.jpg', 'KakaoTalk_20220305_205656630.jpg', 'KakaoTalk_20220305_205659181.jpg', 'KakaoTalk_20220305_205658658.jpg', 'KakaoTalk_20220305_205702855.jpg', 'KakaoTalk_20220305_205701290.jpg', 'KakaoTalk_20220305_205700760.jpg', 'KakaoTalk_20220305_205659711.jpg', 'KakaoTalk_20220305_205702338.jpg', 'KakaoTalk_20220305_205701840.jpg']
['IMG_7340.JPG', 'IMG_7335.JPG', 'IMG_7345.JPG', 'IMG_7344.JPG', 'IMG_7342.JPG', 'IMG_7346.JPG', 'IMG_7336.JPG', 'IMG_7343.JPG', 'IMG_7347.JPG', 'IMG_7341.JPG']


In [2]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/content/drive/MyDrive/dataset/train/"
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


VALIDATION_DIR = "/content/drive/MyDrive/dataset/validation/"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
TRAINING_DIR,
target_size=(150, 150),
class_mode='categorical',
batch_size=100
)

validation_generator = validation_datagen.flow_from_directory(
VALIDATION_DIR,
target_size=(150, 150),
class_mode='categorical',
batch_size=100
)

model = tf.keras.models.Sequential([
  # Note the input shape is the desired size of the image 150x150 with 3 bytes color
  # This is the first convolution
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  # The second convolution
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  # The third convolution
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  # The fourth convolution
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  # Flatten the results to feed into a DNN
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.5),
  # 512 neuron hidden layer
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax')
])


model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=50, steps_per_epoch=10, validation_data = validation_generator, verbose = 2, validation_steps=3)

model.save("model.h5")

Found 2102 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                       

In [4]:

#로컬에 저장하기
from google.colab import files
files.download('model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from keras.preprocessing import image
import numpy as np

model = load_model('model.h5')
model.summary()

# predict = model.predict(validation_generator, steps=5)
# print(validation_generator.class_indices)
# print("예측값: ", predict)
######################################################
TEST_DIR = "/content/drive/MyDrive/dataset/test/"
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size = (150,150),
    class_mode='categorical',
)
img = image.load_img("/content/drive/MyDrive/dataset/test/KakaoTalk_20220315_150133955.jpg", target_size=(150,150))
img2 = image.img_to_array(img)
img2 = np.expand_dims(img2, axis = 0)

predict1 = model.predict(img2)
print(test_generator.class_indices)
print("예측값1: ", predict1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0